In [1]:
using WAV, Knet

In [2]:
music_sample_size = 479232
sample_frequency = 16000
batch_size = 6

function load_data()
    X = Array{Array{Float32,1},1}()
    for musician in ["satriani"]
        for sample in readdir("/mnt/data/other/music_samples/processed/$(musician)")
            values_raw, fq = wavread("/mnt/data/other/music_samples/processed/$(musician)/$(sample)")
            if length(values_raw[:,1]) >= music_sample_size
                push!(X, values_raw[:,1][1:music_sample_size])
            end
        end
    end
    Array{Float32,2}(hcat(X...))
end

X = load_data()

479232×95 Array{Float32,2}:
 0.0         0.1377      -0.0639058   …  -0.0993988   -0.104984   
 0.0         0.104648    -0.0940275      -0.0798669   -0.115879   
 0.0         0.0352794   -0.206671       -0.0525529   -0.125553   
 0.0         0.0253609   -0.241523       -0.0338755   -0.135533   
 0.0        -0.0348216   -0.282113       -0.00335704  -0.137852   
 0.0         0.130985    -0.349956    …   0.0128178   -0.136113   
 0.0         0.159062    -0.251076        0.024781    -0.143345   
 0.0         0.160314    -0.0500198       0.038667    -0.151585   
 0.0         0.0520341    0.0230415       0.0429701   -0.149998   
 0.0         0.156652    -0.0125431       0.0480361   -0.153569   
 0.0         0.0842311   -0.0633564   …   0.0591754   -0.162877   
 0.0        -0.0530106   -0.00817896      0.0581378   -0.167699   
 0.0         0.0857265    0.120518        0.0435194   -0.17011    
 ⋮                                    ⋱                           
 0.239967   -0.180181    -0.082308

We don't need labels anymore, since we only need to label "fake" and "real" and all the samples are real

In [4]:
function create_discriminator()
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(samples*2/(3*512)),samples),
        xavier(Float32,Int64(samples/1024),Int64(samples*2/(3*512))), 
        xavier(Float32,Int64(samples/2048),Int64(samples/1024)), 
        xavier(Float32,2,Int64(samples/2048))
    ]
    Biases = [
        zeros(Float32,samples*2/(3*512),1),
        zeros(Float32,samples/1024,1),
        zeros(Float32,samples/2048,1),
        zeros(Float32,2,1)
    ]
    Model = Any[]
    for i = 1:length(Weights)
        push!(Model, Weights[i])
        push!(Model, Biases[i])
    end
    Model
end

function create_generator()
    granularity = 64
    oscilators = 80
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity)),
        xavier(Float32,Int64(oscilators * granularity / 2), Int64(oscilators * granularity)), 
        xavier(Float32,Int64(oscilators * granularity / 4), Int64(oscilators * granularity / 2)), 
        xavier(Float32,music_sample_size, Int64(oscilators * granularity / 4)),
        #xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity / 4))
    ]
    Biases = [
        xavier(Float32,Int64(oscilators * granularity), 1),
        xavier(Float32,Int64(oscilators * granularity / 2), 1),
        xavier(Float32,Int64(oscilators * granularity / 4), 1),
        xavier(Float32,music_sample_size, 1)
        #xavier(Float32,Int64(oscilators * granularity), 1)
    ]
    Model = Any[]
    for i = 1:length(Weights)
        push!(Model, Weights[i])
        push!(Model, Biases[i])
    end
    Model
end

create_generator (generic function with 1 method)

In [6]:
function discriminate(Model,X)
    for i=1:2:length(Model) - 2
        X = tanh.(Model[i]*X .+ Model[i+1])
    end
    labels = tanh.(Model[end - 1]*X .+ Model[end])
    return labels
end
#=
function generate(Model, batch_size)
    Z = Array(randn(Float32, music_sample_size, batch_size))
    for i=1:2:length(Model) - 2
        Z = tanh.(Model[i]*Z .+ Model[i+1])
    end
    labels = tanh.(Model[end - 1]*Z .+ Model[end])
    return labels
end
=#
granularity = 64
oscilators = 80
function generate(Model, Z=nothing; nr_smaples=6)
    if Z == nothing
       Z = Array(xavier(Float32, granularity*oscilators, nr_smaples)) 
    end
    for i=1:2:length(Model) - 2
        Z = tanh.(Model[i]*Z .+ Model[i+1])
    end
    Z = tanh.(Model[end - 1]*Z .+ Model[end])
    Z
    #=
    samples = Array(zeros(Float32, music_sample_size, 1))
    
    vals = [] 
    for sample_nr=0:oscilators:oscilators*(granularity - 1)
        push!(vals, Z[(sample_nr + 1):(sample_nr + oscilators)])        
    end
    
    for si=1:music_sample_size
        oi = 1
        for v in vals[Int64(round(1 + si/music_sample_size * (granularity -1) ))]
            samples[si] += sin(si*(oi^(2.3))/16000)*v
            oi += 1
        end
        samples[si] = samples[si]/oscilators
    end
    println("Trying to deliver samples")
    hcat([]...)
    =#
end

generate (generic function with 2 methods)

In [7]:
D = create_discriminator()
G = create_generator()

loss_d(D,X,X_fake) = -mean((discriminate(D,X))+(1-discriminate(D,X_fake)))/2
∇d = grad(loss_d)

loss_g(G,D) = -mean(log.( discriminate(D, generate(G, nr_smaples=2)) ))
∇g = grad(loss_g)

optimizers_d = optimizers(D, Adam)
optimizers_g = optimizers(G, Adam)


function save_samples(epoch)
    samples = generate(G,nr_smaples=2)
    for i=1:length(samples[1,:])
        wavwrite(samples[:,i], 16000, "/mnt/data/other/music_samples/generated/epoch-$(epoch)-nr-$(i).wav")
        println("Generated a sample !")
    end
end


for epoch=1:20
    println("Started training for epoch: $(epoch)")
    
    for i=1:5:(length(X) - 4)
        
        update!(D, ∇d(D,X[:, i:i+5],generate(G, nr_smaples=2)) , optimizers_d)
        
        update!(G, ∇g(G,D) , optimizers_g)
        
    end
    println("Printing some random neurons from G: $(G[1][12])  | $(G[2][77]) | $(G[3][22]) | $(G[4][19]) And the sum of two layers: $(sum(G[1])), $(sum(G[6]))")
    println("Printing some random neurons from D: $(D[1][12])  | $(D[2][77]) | $(D[3][22]) | $(D[4][19]) And the sum of two layers: $(sum(D[1])), $(sum(D[6]))")
    println("Finished training epoch: $(epoch)")
    println("Got accuracy: $(accuracy(D, Dtrain, discriminate)) , on the training set")
    save_samples(epoch)
    flush(STDOUT)
end

Started training for epoch: 1


LoadError: [91mDimensionMismatch("dimensions must match")[39m

For how grad works, check: https://github.com/denizyuret/AutoGrad.jl/blob/master/src/core.jl
Reference GAN: https://github.com/George3d6/GAN-70-Lines-of-Julia